Load the three embeddings and the preprocessed_useful_data.csv to skip Preprocessing and embedding

# Preprocessing

In [ ]:
import pandas as pd
import re
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import json

# Download required NLTK data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

# Load the Excel file
df = pd.read_excel("/content/useful_data.xlsx")
df = df.rename(columns={'faq [json]': 'faq_json'})
df=df.rename(columns={'Category: Name': 'Catagory_name'})
df=df.rename(columns={'Variant Price': 'Variant_price'})
df=df.rename(columns={'how_to_step ': 'how_to_step'})

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


FileNotFoundError: [Errno 2] No such file or directory: '/content/useful_data.xlsx'

FileNotFoundError: [Errno 2] No such file or directory: '/content/useful_data.xlsx'

In [ ]:
df.head()

In [ ]:
print('(2999, 11)')
# Remove duplicates based on 'Title'
df = df.drop_duplicates(subset=['Title'], keep='first')
print(df.shape)


In [ ]:
df.count_nan = df.isna().sum()
print(df.count_nan)

In [ ]:
# --- Data Cleaning ---
# Fill missing values with "No data"
for col in df.columns:
    df[col] = df[col].fillna("No data")



In [ ]:
df.count_nan = df.isna().sum()
print(df.count_nan)

In [ ]:
df.head()

Text Preprocessing
- Lowercases text, removes HTML tags & punctuation
- Converts list-like text data into a cleaned string.
- Extracts questions & answers from JSON FAQ lists





In [ ]:
import json
import pandas as pd
import re
from bs4 import BeautifulSoup

# --- Text Preprocessing ---
def preprocess_text(text):
    """Lowercases text, removes HTML tags & punctuation."""
    if isinstance(text, str):
        text = text.lower()
        text = BeautifulSoup(text, "html.parser").get_text()  # Remove HTML
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        return text.strip()
    return ""

def preprocess_list(lst):
    """Converts list-like text data into a cleaned string."""
    if isinstance(lst, str):
        try:
            lst = json.loads(lst)  # Convert from string to actual list
        except:
            return ""
    if isinstance(lst, list):
        return " ".join([preprocess_text(str(item)) for item in lst])  # Convert list to text
    return ""

# --- Handle JSON Data (FAQs) ---
def safe_json_parse(json_str):
    """Safely parses JSON string, handling 'No data' & malformed JSON."""
    try:
        json_str = str(json_str).strip()
        if json_str.lower() == "no data":  # Handle "No data" cases
            return []
        return json.loads(json_str) if json_str else []  # Load valid JSON
    except:
        return []  # Return empty list for errors

def combine_faq(faq_list):
    """Extracts questions & answers from JSON FAQ lists."""
    text = []
    if isinstance(faq_list, list):
        for faq in faq_list:
            if isinstance(faq, dict):
                question = faq.get('question', '')  # Avoid KeyError
                answer = faq.get('answer', '')
                if question and answer:
                    text.append(f"{question} {answer}")
    return " ".join(text)

# --- Column Lists ---
text_cols = ['Title', 'Description', 'Type', 'Tags', 'Catagory_name']
list_cols = ['key_benefits_list', 'skin_concerns', 'how_to_step']
json_cols = ['faq_json']  # Correct column name

# Apply text preprocessing
for col in text_cols:
    df[col] = df[col].apply(preprocess_text)

# Apply list preprocessing
for col in list_cols:
    df[col] = df[col].apply(preprocess_list)

# Handle FAQs
df['faq_json'] = df['faq_json'].apply(safe_json_parse)  # Parse JSON safely
df['faq_combined'] = df['faq_json'].apply(combine_faq)  # Extract Q&A
df['faq_combined'] = df['faq_combined'].apply(preprocess_text)  # Clean text


In [ ]:
df.head()
# makin gsure json is working

Combine Description and Tags

In [ ]:
# Select columns to concatenate
cols_to_concat = ['Description', 'Tags']

# Convert selected columns to string and concatenate
df['Description_and_Tags'] = df[cols_to_concat].astype(str).agg(' '.join, axis=1)

df.head()
# Save the preprocessed data
df.to_csv("preprocessed_useful_data.csv", index=False)
# print("Preprocessing complete! Saved to preprocessed_useful_data.csv")


In [ ]:
df.head()
# Seeing if concatination is working

# Embedding Creation



---


I will create three embeddings based on Description, Tags and Description_and_Tags because I am still experimenting.

- 'Tags' have all the  key words which uniquely describe the product.

- 'Description' column has text in natural language and will add context

- Combination of both will be the best descriptor of the product.(i used this for similarity search during quering)


---


Model for embeddings creation- **all-mpnet-l6-v2**.

Reason-
- Best in class accuracy
- Sentence Transformer for long sequential data
- High precision in semantic search
- 768 dimensions dense vectors
  
Used Kaggle gpu to speedup the process

---



In [ ]:
# #This is commented out to stop creating the embeddings again



# !pip install sentence-transformers
# from sentence_transformers import SentenceTransformer
# # !pip uninstall torch torchvision sentence-transformers -y

# import pandas as pd
# import numpy as np


# # 1. Load your data
# df_embedding = pd.read_csv("/content/preprocessed_useful_data.csv")
# # df_embedding=df.copy

# from tqdm import tqdm

# # 2. Initialize the Sentence Transformer model
# model = SentenceTransformer('all-mpnet-base-v2')

# # 3. Generate embeddings with a loading bar
# embeddings_Description = np.array([
#     model.encode(text) for text in tqdm(df_embedding['Description'].tolist(), desc="Generating Embeddings")
# ])


# # 4. 'embeddings' is now a numpy array containing the embeddings for each product
# print(embeddings_Description.shape)

# # Save the embeddings for future use
# np.save('embeddings_Description.npy', embeddings_Description)







In [ ]:
# import torch
# import numpy as np
# import pandas as pd
# from tqdm import tqdm
# from sentence_transformers import SentenceTransformer

# # Load Dataset
# df_embedding = pd.read_csv("/kaggle/input/embedding-description/preprocessed_useful_data.csv")  # Update the path if needed

# #  Ensure No Missing Values
# df_embedding['Description'] = df_embedding['Description'].fillna("")

# # Check if GPU is Available
# device = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using device: {device}")

# #  Load Sentence Transformer Model on GPU
# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device=device)

# # Generate Embeddings Efficiently with GPU Support
# embeddings_Description = model.encode(
#     df_embedding['Description'].tolist(),  # Convert descriptions to list
#     batch_size=32,  # Adjust batch size based on GPU memory
#     show_progress_bar=True,  # Show a progress bar in Kaggle
#     convert_to_numpy=True,  # Return as NumPy array
#     device=device  # Use GPU if available
# )

# # Print Shape & Save Embeddings
# print("Embeddings Shape:", embeddings_Description.shape)
# np.save('/kaggle/working/embeddings_Description.npy', embeddings_Description)  # Saves the file in Kaggle’s output directory



# For quick Run, Start here

In [ ]:
# Purposfully left empty
# For quick run make sure that you Load the three embeddings and the preprocessed_useful_data.csv
# The cells above can be skipped.
# Run this cell and all the cells below. Put the query in the "Input query here" cell


In [ ]:
# All the liberaries for faster

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

!pip install -q -U langchain langchain_core langchain_openai
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate



# Merging the preprocessed_useful_data and all the three embeddings

The reason for merging the product data and embedding is to store them in the dataframe together and store similarity_score corresponding to them. This makes retreival easier.

In [ ]:

df_merged = pd.read_csv("/content/preprocessed_useful_data.csv")
embeddings_Description = np.load('/content/embeddings_Description.npy')
embeddings_Tags = np.load('/content/embeddings_Tags.npy')
embeddings_Description_and_Tags = np.load('/content/embeddings_Description_and_Tags.npy')

In [ ]:
df_merged['embeddings_Description'] = embeddings_Description.tolist()
df_merged['embeddings_Tags'] = embeddings_Tags.tolist()
df_merged['embeddings_Description_and_Tags'] = embeddings_Description_and_Tags.tolist()
df_merged.head()

,Title,Description,Type,Tags,URL,Catagory_name,Variant_price,faq_json,how_to_step,key_benefits_list,skin_concerns,Description_and_Tags,embeddings_Description,embeddings_Tags,embeddings_Description_and_Tags
0,melaglow rich cream,melaglow rich cream is a powerful depigmentati...,skin cream,abbott age spots ageing ahabha range antiagein...,https://www.clinikally.com/products/melaglow-r...,skin care,389.0,NaN,cleanse your face thoroughly apply a small amo...,lightens pigmentation dark spots and blemishes...,antiageing antipigmentation day cream damage p...,melaglow rich cream is a powerful depigmentati...,"[0.06199994310736656, 0.010653379373252392, 0....","[0.031957048922777176, 0.03539339452981949, 0....","[0.060895293951034546, 0.013484464026987553, 0..."
1,new follihair tablet bottle,new follihair tablet bottle is a comprehensive...,hair supplement,abbott abbott healthcare amino acids daily sup...,https://www.clinikally.com/products/follihair-...,hair care,599.0,NaN,one tablet daily can be taken or as directed b...,encourages healthy hair growth and reduces hai...,NaN,new follihair tablet bottle is a comprehensive...,"[0.02432352304458618, -0.028881017118692398, -...","[-0.002312002470716834, 0.03493695706129074, 0...","[0.02481081150472164, -0.0052250344306230545, ..."
2,episoft ac moisturiser with sunscreen spf 30,episoft ac moisturiser with sunscreen spf 30 i...,sunscreen,acne care acne prone body cream ctmpspr d pant...,https://www.clinikally.com/products/glenmark-e...,skin care,469.0,NaN,apply liberally on the face neck and area of t...,episoft ac moisturizes and protects as a sunsc...,day cream daily care,episoft ac moisturiser with sunscreen spf 30 i...,"[-0.014883074909448624, -0.024709103628993034,...","[0.02320794388651848, 0.0512845516204834, 0.01...","[0.007932344451546669, 0.003804840613156557, -..."
3,neutriderm hair enhancer lotion,neutriderm hair enhancer lotion powered by sab...,hair lotion,bald patches baldness benzoic acid biotin ctcp...,https://www.clinikally.com/products/neutriderm...,hair care,3749.0,NaN,apply four sprays directly onto the scalp gent...,decreases hair breakage by 90 within one month...,NaN,neutriderm hair enhancer lotion powered by sab...,"[0.020290618762373924, -0.057306721806526184, ...","[0.027835577726364136, 0.04846222698688507, -0...","[0.033620115369558334, -0.02959146909415722, -..."
4,epique intensive cellular repair serum,epique intensive cellular repair serum is infu...,skin serum,ageing algae extract antiageing arginine boswe...,https://www.clinikally.com/products/epique-int...,skin care,5699.0,NaN,warm a few drops of serum between your fingert...,provides effective hydration to the skin boost...,antiageing wrinkles,epique intensive cellular repair serum is infu...,"[0.02807873673737049, 0.0064684245735406876, -...","[0.0595456138253212, 0.07383878529071808, 0.00...","[0.04035727679729462, 0.03609625622630119, -0...."


In [ ]:
df_merged.shape

(2653, 15)

In [ ]:
# prompt: drop Description_and_Tags

df_merged = df_merged.drop(columns=[	'Type',		'Catagory_name',		'how_to_step',	'key_benefits_list','faq_json',	'skin_concerns',	'Description_and_Tags'])
df_merged.head(1)

,Title,Description,Tags,URL,Variant_price,embeddings_Description,embeddings_Tags,embeddings_Description_and_Tags
0,melaglow rich cream,melaglow rich cream is a powerful depigmentati...,abbott age spots ageing ahabha range antiagein...,https://www.clinikally.com/products/melaglow-r...,389.0,"[0.06199994310736656, 0.010653379373252392, 0....","[0.031957048922777176, 0.03539339452981949, 0....","[0.060895293951034546, 0.013484464026987553, 0..."


In [ ]:
# #download df_merged

# from google.colab import files
# df_merged.to_csv('product_context_and_embeddings.csv', encoding = 'utf-8-sig')
# files.download('product_context_and_embeddings.csv')


# Input Query Here  :)

In [ ]:

# Product Recommendation Queries
# user_query = "Recommend a moisturizer for oily skin under 1000 rupees."

# General Queries
# user_query="How often should I exfoliate my face?"

# Product Information Queries( not asking for recommendation rather information about a product)
# user_query="Benifits of Episoft AC Moisturiser"

# Unrelated Queries
user_query="What is the capital of Argentina?"


'''

# General Queries
general_queries = [
    "What is the best way to take care of sensitive skin?"
    "How often should I exfoliate my face?"
    "Are there any natural remedies for acne?"
]
# Product Recommendation Queries
product_recommendation_queries = [
    "Recommend a moisturizer for oily skin under $1200."
    "I need a lightweight serum for anti-aging."
    "Suggest a fragrance-free cleanser for sensitive skin."
]

# Product Information Queries
product_information_queries = [
    "Does the Episoft AC Moisturiser with Sunscreen SPF 30 brand moisturizer contain parabens?"
]
# Unrelated Queries
unrelated_queries = [
    "Who won the FIFA World Cup in 2022?"
    "How do I cook spaghetti carbonara?"
    "What is the capital of Argentina?"
]


'''



'\n\n# General Queries\ngeneral_queries = [\n    "What is the best way to take care of sensitive skin?"\n    "How often should I exfoliate my face?"\n    "Are there any natural remedies for acne?"\n]\n# Product Recommendation Queries\nproduct_recommendation_queries = [\n    "Recommend a moisturizer for oily skin under $1200."\n    "I need a lightweight serum for anti-aging."\n    "Suggest a fragrance-free cleanser for sensitive skin."\n]\n\n# Product Information Queries\nproduct_information_queries = [\n    "Does the Episoft AC Moisturiser with Sunscreen SPF 30 brand moisturizer contain parabens?"\n]\n# Unrelated Queries\nunrelated_queries = [\n    "Who won the FIFA World Cup in 2022?"\n    "How do I cook spaghetti carbonara?"\n    "What is the capital of Argentina?"\n]\n\n\n'

# Querying

The Idea is to
- fetch user query,  
- pass it to sentence transformer model (all mpnet base l6 v2),  
- get the vector as query_vector, perform cosine similarity,  
- calculate similarity_score=max(query and D, query and T, query and D_and_T)  
- Sort on the basis of similarity score and return the top 7 vector as a df called top_7_similar_df which contains some columns of merged_df and the similarity_score as an extra column  




---


- The function get_top_7_similar
 - takes in user_query(string) and dataframe (with product information columns as well as the 3 embedding columns),
 - creates a query_vector,
 - calculates the similarity score between query vector all the three embedding vectors individually.

 I experimented with multiple combinations of similarity_score calculation like average(score1, score2, score3), max(score1, score2, score3), different weights, but using sim_description_and_tags gave the best results.


---



In [ ]:
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# Load the sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')

def get_top_7_similar(user_query, df_merged):
    # Generate query vector
    query_vector = model.encode([user_query])[0]

    # Calculate cosine similarity for each embedding
    sim_description = cosine_similarity([query_vector], np.stack(df_merged['embeddings_Description'].values))[0]
    sim_tags = cosine_similarity([query_vector], np.stack(df_merged['embeddings_Tags'].values))[0]
    sim_description_and_tags = cosine_similarity([query_vector], np.stack(df_merged['embeddings_Description_and_Tags'].values))[0]

    # Calculate similarity score
    # similarity_score = np.maximum(sim_description, sim_tags, sim_description_and_tags)
    similarity_score = ( sim_description_and_tags )

    # Add similarity score to the DataFrame
    df_merged['similarity_score'] = similarity_score

    # Sort by similarity score and get top 50
    top_7_similar_df = df_merged.sort_values('similarity_score', ascending=False).head(7)

    return top_7_similar_df

# Example usage
# user_query = "Recommend a moisturizer for oily skin under 1000 rupees."
top_7_similar_df = get_top_7_similar(user_query, df_merged)




---


preprocessing top_7_similar_df so that it can be passed to llm


*   removing extra columns


---




In [ ]:
top_7_similar_df.head()

,Title,Description,Tags,URL,Variant_price,embeddings_Description,embeddings_Tags,embeddings_Description_and_Tags,similarity_score
2254,website difference,no data,exclude_rebuy location59840397475outofstock lo...,https://www.clinikally.com/products/website-di...,1.0,"[-0.024348972365260124, 0.1014820858836174, 0....","[-0.0055386838503181934, -0.00749971391633153,...","[-0.008828320540487766, -0.0002122543955920264...",0.132493
653,tango hair serum,tango hair serum is a nutrientrich leavein con...,antioxidant antioxidant properties antioxidant...,https://www.clinikally.com/products/tango-hair...,469.0,"[0.05240122228860855, -0.03304005414247513, -0...","[0.032857708632946014, 0.017650388181209564, -...","[0.0595402792096138, -0.006008760072290897, -0...",0.129446
742,sesderma ferulac dubai lip balm,sesderma ferulac dubai lip balm is a brighteni...,chapped lips clinikally luxe daily care dark l...,https://www.clinikally.com/products/sesderma-f...,2000.0,"[0.044806960970163345, -0.05022333934903145, -...","[0.010190451517701149, -0.017891310155391693, ...","[0.03172077611088753, -0.04617605730891228, -0...",0.105293
1148,densita everyday clarifying shampoo,densita everyday clarifying shampoo is a sulfa...,antifrizz argan oil damaged frizzy hair elimi...,https://www.clinikally.com/products/densita-ev...,379.0,"[0.002997255651280284, -0.02851276472210884, -...","[-0.006300286389887333, 0.026988377794623375, ...","[0.01001692097634077, -0.021604612469673157, -...",0.105205
495,kerawash hair conditioning shampoo,kerawash hair conditioning shampoo gently clea...,beta vulgaris root extract ceramide ceramides ...,https://www.clinikally.com/products/ipca-keraw...,379.0,"[-0.007600733079016209, -0.0033883387222886086...","[0.006460240576416254, 0.009253977797925472, -...","[0.011470174416899681, -0.004542358219623566, ...",0.096553


In [ ]:
# prompt: drop Description_and_Tags

top_7_similar_df = top_7_similar_df.drop(columns=[	'embeddings_Description',	'embeddings_Tags',	'embeddings_Description_and_Tags'])


In [ ]:
top_7_similar_df.head(17)

,Title,Description,Tags,URL,Variant_price,similarity_score
2254,website difference,no data,exclude_rebuy location59840397475outofstock lo...,https://www.clinikally.com/products/website-di...,1.0,0.132493
653,tango hair serum,tango hair serum is a nutrientrich leavein con...,antioxidant antioxidant properties antioxidant...,https://www.clinikally.com/products/tango-hair...,469.0,0.129446
742,sesderma ferulac dubai lip balm,sesderma ferulac dubai lip balm is a brighteni...,chapped lips clinikally luxe daily care dark l...,https://www.clinikally.com/products/sesderma-f...,2000.0,0.105293
1148,densita everyday clarifying shampoo,densita everyday clarifying shampoo is a sulfa...,antifrizz argan oil damaged frizzy hair elimi...,https://www.clinikally.com/products/densita-ev...,379.0,0.105205
495,kerawash hair conditioning shampoo,kerawash hair conditioning shampoo gently clea...,beta vulgaris root extract ceramide ceramides ...,https://www.clinikally.com/products/ipca-keraw...,379.0,0.096553
2388,novology hydrating sheer sunscreen spf 50 pa,novology hydrating sheer sunscreen spf 50 pa i...,brighten skin broad spectrum sunscreen damaged...,https://www.clinikally.com/products/novology-h...,549.0,0.093402
132,rene furterer lumicia illuminating shine shampoo,rene furterer lumicia illuminating shine shamp...,cleanse hair clinikally luxe conditioning sham...,https://www.clinikally.com/products/rene-furte...,1500.0,0.091464


In [ ]:
# # prompt: download the top_7_similar_df dataframe as csv

# top_7_similar_df.to_csv('top_7_similar_df.csv', encoding = 'utf-8-sig')
# # files.download('top_7_similar_df.csv')


## Rough Prompts

I want through multiple iterations of prompts, improving something each time. Feel free to Skip to the next section :)

In [ ]:
# I already have the top_50_similar_df (acting as product description for the 50 most similar product based on user query) and I want to pass top_50_similar_df, the innitial user query and the prompt to groq (model mistral 8x7B) to get a response.

# For General queries
#  If the query is a general query and unrelated to products, but still in the the domain of ['skin care', 'hair care', 'vitamins  supplements', 'handheld devices', 'vision care', 'health care', 'gift cards','skin cream', 'hair supplement', 'sunscreen', 'hair lotion',
#        'skin serum', 'hair cleanser', 'skin lotion', 'skin cleanser',
#        'skin supplement', 'hair mask', 'hair serum', 'skin emulsion',
#        'skin gel', 'scalp solution', 'skin patch', 'hair oil',
#        'hair care combo', 'skin solution', 'hair cream', 'hair colour',
#        'lip care', 'skin care combo', 'face mask', 'health supplement',
#        'skin powder', 'skin ointment', 'skin oil', 'service', 'devices',
#        'hair gel', 'eye solution', 'medicine', 'gift card', 'skin wipes'].
#  i want to use the llm to answer in a concise way (not more than 100 words). If the general query is out of the scope of these topics do not answer it.

# For Product related queries
# For queries which in which we have to recommend a product
# I will pass the top_50_similar_df dataframe for the product description and the user query.
# I want to find the most similar product based on the ['Description', 'Tags', key_benefits_list,	skin_concerns, faq_combined, Variant_price, similarity_score] columns in top_50_similar_df . Give more weightage to textual columns like [key_benefits_list,	skin_concerns , description] than numerical columns like similarity_score.
# If the user mentions a price range then give extra focus on the Variant_price column. This column will act as a filter and important in case the user mentions a price range. Please still consider the other text based columns for finding the most relevent product but use the Variant_price column as a filter.
# Based on the user query, recommend them two products. Add the product price(from Variant_price), the URL to buy the product (from URL column in top_50_similar_df),and a small description of the two products(one line), I also want you to add four other products( not the same as the two recommmended earlier) as "similar products"



# ps donot truncate any column of this dataframe. consider the full text from all the textual columns.

In [ ]:
# prompt1='''You are an expert skincare and healthcare advisor. Your task is to provide helpful information or product recommendations based on user queries. Here are your instructions:

# 1. For general queries related to skincare, haircare, health supplements, or other related topics:
#    - Provide a concise answer (maximum 100 words).
#    - Only answer if the query is within the specified domains.
#    - If the query is outside these domains, politely inform the user that you can't assist with that topic.

# 2. For product-related queries:
#    - Analyze the provided top_50_similar_df dataframe, focusing on the 'Description', 'Tags', 'key_benefits_list', 'skin_concerns', 'faq_combined', 'Variant_price', and 'similarity_score' columns.
#    - Give more weight to textual columns like 'key_benefits_list', 'skin_concerns', and 'Description' when determining relevance.
#    - If a price range is mentioned, use the 'Variant_price' column as a filter while still considering other relevant factors.
#    - Recommend two products that best match the query, including:
#      * Product name
#      * Price (from 'Variant_price' column)
#      * URL to purchase (from 'URL' column)
#      * A brief one-line description
#    - Suggest four additional "similar products" (different from the two main recommendations).

# Provide a clear, concise response that directly addresses the user's query or need.

# User Query: [Insert user query here]

# [Insert top_50_similar_df data here]

# Based on the above information, please provide your response:
# '''

In [ ]:
# prompt2 = ChatPromptTemplate.from_messages([
#     ("system",
#      """You're Clinikally's expert skincare advisor. Follow these rules:

#      1. GENERAL QUERIES (non-product):
#      - Concise answer (<100 words)
#      - Domain: Skincare/Haircare/Supplements
#      - Medical disclaimers where needed

#      2. PRODUCT REQUESTS:
#      Analyze these columns from top_50_similar_df:
#      [Description (30% weight), Tags (20%), key_benefits_list (25%),
#      skin_concerns (15%), faq_combined (10%)]

#      Recommendations must include:
#      - 2 Primary Picks: Highest relevance + price match
#      - 4 Alternatives: Similar efficacy + varying price points
#      - Always include URL and price

#      Format:
#      「Recommended Products」
#      1. [Name] ({Price}₹)
#         • URL: {URL}
#         • Key Benefit: [Most relevant benefit]

#      「Alternative Options」
#      a) [Name] ({Price}₹) - [Differentiating feature]
#      ..."""),

#     ("human", "Query: {user_query}\n\nProduct Data:\n{product_info}")
# ])


In [ ]:
# prompt3='''You are an expert skincare and healthcare advisor. Your task is to provide helpful information or product recommendations based on user queries. Here are your instructions:

# 1. For general queries related to skincare, haircare, health supplements, or other related topics:
#    - Provide a concise answer (maximum 100 words).
#    - Only answer if the query is within the specified domains.
#    - If the query is outside these domains, politely inform the user that you can't assist with that topic.

# 2. For product-related queries:
#    - Analyze the provided top_50_similar_df dataframe, focusing on the 'Description', 'Tags', 'key_benefits_list', 'skin_concerns', 'faq_combined', 'Variant_price', and 'similarity_score' columns.
#    - Use the following weighting for relevance scoring:
#      * Description: 30%
#      * key_benefits_list: 25%
#      * skin_concerns: 20%
#      * Tags: 15%
#      * faq_combined: 10%
#    - If a price range is mentioned, use the following filtering rules:
#      * "Under ₹X" → Include products where Variant_price ≤ X
#      * "Between ₹X and ₹Y" → Include products where X ≤ Variant_price ≤ Y
#    - Recommend two products that best match the query, including:
#      * Product name
#      * Price (from 'Variant_price' column)
#      * URL to purchase (from 'URL' column)
#      * A brief one-line description
#    - Suggest four additional "similar products" (different from the two main recommendations).

# 3. If the query mentions medical conditions like eczema, psoriasis, etc., include this disclaimer: "For personalized medical advice, please consult a dermatologist."

# Remember to consider the full text from all columns without truncation. Provide a clear, concise response that directly addresses the user's query or need.

# User Query: [Insert user query here]

# [Insert top_50_similar_df data here]

# Based on the above information, please provide your response:

# Recommended Products

# 1️⃣ [Product Name] (₹[Price])
# - Purchase: [URL]
# - Key Benefit: [Brief one-line description]

# 2️⃣ [Product Name] (₹[Price])
# - Purchase: [URL]
# - Key Benefit: [Brief one-line description]

# Alternative Options

# a) [Product Name] (₹[Price]) - [Differentiating feature]
# b) [Product Name] (₹[Price]) - [Differentiating feature]
# c) [Product Name] (₹[Price]) - [Differentiating feature]
# d) [Product Name] (₹[Price]) - [Differentiating feature]
# '''

In [ ]:
# prompt4 = ChatPromptTemplate.from_messages([
#     ("system", """You are an expert skincare and healthcare advisor. Your task is to provide helpful information or product recommendations based on user queries. Here are your instructions:
# To decide if a query is a general query or a product request, consider the following:
# Keyword Analysis:

# Create two lists of keywords:
# Product-related: "recommend","suggest", "buy", "price", "under", "for", "moisturizer", "shampoo", etc.
# General skincare: "how to", "what causes", "treatment for", "prevent", etc.

# Check if the query contains words from either list.

# If product-related keywords are found, flag as likely product query.

# If general skincare keywords are found, flag as likely general query.

# Query Structure Analysis:

# Product queries often include specific attributes or price ranges, while general queries are typically more open-ended. For example:

# Product query: "Recommend a moisturizer for oily skin under 1000 rupees"

# General query: "How can I treat acne?"

# After identifying the query type, follow these rules:

# 1. For general queries related to skincare, haircare, health supplements, or other related topics:
#    - Provide a concise answer (maximum 100 words).
#    - Only answer if the query is within the specified domains.
#    - If the query is outside these domains, politely inform the user that you can't assist with that topic.
#     Format for general queries ONLY(donot use this for product related queries)
#       Here's what I found on the web
#       concise answer with the source of the information
#     donot give product recommendation for general queries. Stop after a concise 3 line answer(100 words)
#     examples of general queries

# 2. For product-related queries:
#    - Analyze the provided top_7_similar_df dataframe, focusing on the 'Description', 'Tags', 'Variant_price', and 'similarity_score' columns.
#    - Give more weight to textual columns like 'Description' and 'Tags' when determining relevance.
#    - If a price range is mentioned, use the 'Variant_price' column as a filter while still considering other relevant factors.
#    - Recommend two products that best match the query, including:
#      * Product name
#      * Price (from 'Variant_price' column)
#      * URL to purchase (from 'URL' column)
#      * A brief one-line description
#    - Suggest four additional "similar products" (different from the two main recommendations).
#     Format for product related queries ONLY(donot use this for general queries)
#         Recommended Products

#         1️⃣ [Product Name] (₹[Price])
#         - Purchase: [URL]
#         - Key Benefit: [Brief one-line description]

#         2️⃣ [Product Name] (₹[Price])
#         - Purchase: [URL]
#         - Key Benefit: [Brief one-line description]

#         Alternative Options

#         a) [Product Name] (₹[Price]) - [Differentiating feature]
#         b) [Product Name] (₹[Price]) - [Differentiating feature]
#         c) [Product Name] (₹[Price]) - [Differentiating feature]
#         d) [Product Name] (₹[Price]) - [Differentiating feature]
# Remember to consider the full text from all columns without truncation. Provide a clear, concise response that directly addresses the user's query or need.

# Based on the above information, please provide your response


# """),
#     ("human", "User Query: {user_query}\n\nProduct Data:\n{product_info}")
# ])


In [ ]:
# prompt5 = ChatPromptTemplate.from_messages([
#     ("system", """You are an expert skincare and healthcare advisor. Your task is to first classify the user query and then respond accordingly. Follow these steps:

# ### STEP 1: CLASSIFY THE QUERY
# - **"Product-related"** → If the query asks about **a specific product recommendation, comparison, price, or availability** (e.g., "Recommend a moisturizer for oily skin under 1000 rupees").
# - **"General"** → If the query asks about **skincare, haircare, or health advice** without mentioning a product (e.g., "How can I treat acne?").
# - **"Out-of-scope"** → If the query is unrelated to skincare, haircare, or health, classify it as out-of-scope.

# Use the following rules for classification:
# 1. **Keyword Analysis**:
#    - Product-related: `"recommend", "suggest", "buy", "price", "under", "moisturizer", "shampoo", "best", "cheapest"`, etc.
#    - General skincare: `"how to", "what causes", "treatment for", "prevent", "benefits of"`, etc.
# 2. **Query Structure Analysis**:
#    - Product queries often include **specific attributes or price ranges**.
#    - General queries are **more open-ended**, asking for explanations or causes.

# ---
# ### STEP 2: RESPOND BASED ON CLASSIFICATION

# #### **1️⃣ If "General" Query:**
# - Provide a **concise answer** (max **100 words**).
# - Stay within skincare, haircare, or health topics.
# - **Format:**
#   ```
#   Here's what I found on the web! 😊
#   [Concise 3-line answer (max 50 words)]
#   Hope that helps! Let me know if you need more details.
#   ```
# - **Do NOT** give product recommendations for general queries.

# #### **2️⃣ If "Product-related" Query:**
# - Analyze the provided `top_7_similar_df` dataframe, focusing on:
#   - **Text relevance** → `Description, Tags, key_benefits_list, skin_concerns`
#   - **Price relevance** (if a range is mentioned) → Use `Variant_price` as a filter.
# - **Recommend two products**, including:
#   1. **Product Name**
#   2. **Price** (from `Variant_price` column)
#   3. **URL to purchase** (from `URL` column)
#   4. **A brief one-line key benefit**

# - **Suggest four additional similar products**, different from the two main recommendations.
# - **Format for product-related queries ONLY**:
#   ```
#   Recommended Products

#   1️⃣ [Product Name] (₹[Price])
#   - Purchase: [URL]
#   - Key Benefit: [Brief one-line description]

#   2️⃣ [Product Name] (₹[Price])
#   - Purchase: [URL]
#   - Key Benefit: [Brief one-line description]

#   Alternative Options

#   a) [Product Name] (₹[Price]) - [Differentiating feature]
#   b) [Product Name] (₹[Price]) - [Differentiating feature]
#   c) [Product Name] (₹[Price]) - [Differentiating feature]
#   d) [Product Name] (₹[Price]) - [Differentiating feature]
#   ```

# #### **3️⃣ If "Out-of-Scope" Query:**
# - Politely inform the user that the query is beyond your expertise.
# - Example response:
#   ```
#   I'm sorry, but I can only assist with skincare, haircare, and health-related topics.
#   ```

# ---
# ### **Final Instructions**
# - **Do NOT mix formats** → Use **ONLY** the general query format for general queries and **ONLY** the product query format for product-related queries.
# - **Ensure full text from all relevant columns is considered without truncation**.
# - Provide a **clear, concise response** that **directly answers the user's query**.

# ---
# User Query: "{user_query}"
# Product Data: "{product_info}" (if applicable)

# Now classify the query and respond accordingly.


# """),
#     ("human", "User Query: {user_query}\n\nProduct Data:\n{product_info}")
# ])


# LLM setup and Prompt

I tried multiple LLMs.
- **llama-3.3-70b-versatile**
 - Free and open source (I didnt want to use up the openai key during trial runs.)
 - It offers strong reasoning for long, detailed, or ambiguous queries.
 - Ensures accurate query classification for routing.
 - Trained on web sourced data, so good for general queries.
 - Good natural language responses, so good for chatbot.
 - **Disadvantage**- token limit 6000. Token limit exceed for some queries.

So I Decided to use
 - ***o3-mini ***
  - Has all the benefits of llama-3.3-70b-versatile(except free access)
  - Has high token limit(TPM)
  - Fast, Optimised for RAG application




In [ ]:
# !pip install -q -U langchain langchain_core langchain_groq


# !pip install -q -U langchain langchain_core langchain_openai

In [ ]:
# import os
# from langchain_groq import ChatGroq
# from langchain_core.prompts import ChatPromptTemplate

# # Set your Groq API key
# os.environ["GROQ_API_KEY"] = "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"


# chat = ChatGroq(
#     api_key=os.environ["GROQ_API_KEY"],
#     model_name="llama-3.3-70b-versatile"
# )


Setting up OpenAI environment.

In [ ]:


# import os
# from langchain_openai import ChatOpenAI
# from langchain_core.prompts import ChatPromptTemplate

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "API_KEY"
# I am aware of ways of hiding the key, here there is no need to hid :)

chat = ChatOpenAI(
    model="o3-mini",

)




In [ ]:
def format_product_descriptions(df):
    descriptions = []
    for _, row in df.iterrows():
        desc = " | ".join([f"{col}: {row[col]}" for col in df.columns])
        descriptions.append(desc)
    return "\n".join(descriptions)


The prompt here does the query filteration based on the following


1.   Key word Matching Heuristic
2.   Query Structure Analysis

please see the prompt below for more depth.


In [ ]:
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an expert skincare and healthcare advisor. Your task is to first classify the user query and then respond accordingly. Follow these steps:

### STEP 1: CLASSIFY THE QUERY
- **"Product-related"** → If the query asks about **a specific product recommendation, comparison, price, or availability** (e.g., "Recommend a moisturizer for oily skin under 1000 rupees").
- **"General"** → If the query asks about **skincare, haircare, or health advice** without mentioning a product (e.g., "How can I treat acne?").
- **"Out-of-scope"** → If the query is unrelated to skincare, haircare, or health, classify it as out-of-scope.

Use the following rules for classification:
1. **Keyword Analysis**:
   - Product-related: `"recommend", "suggest", "buy", "price", "under", "moisturizer", "shampoo", "best", "cheapest"`, etc.
   - General skincare: `"how to", "what causes", "treatment for", "prevent", "benefits of"`, etc.
2. **Query Structure Analysis**:
   - Product queries often include **specific attributes or price ranges**.
   - General queries are **more open-ended**, asking for explanations, causes or information.

---
### STEP 2: RESPOND BASED ON CLASSIFICATION

#### **1️⃣ If "General" Query:**
- Provide a **concise answer** (max **100 words**) in a **friendly, conversational tone**.
- Keep it **engaging and natural** while staying within skincare, haircare, or health topics.
- **Format:**
  ```
  Here's what I found on the web! 😊
  [Concise 3-line answer (max 100 words)]
  Hope that helps!
  ```
- **Do NOT** give product recommendations for general queries.

#### **2️⃣ If "Product-related" Query:**
- Analyze the provided `top_7_similar_df` dataframe, focusing on:
  - **Text relevance** → `Description, Tags, key_benefits_list, skin_concerns`
  - **Price relevance** (if a range is mentioned) → Use `Variant_price` as a filter.
- **Recommend two products**, including:
  1. **Product Name**
  2. **Price** (from `Variant_price` column)
  3. **URL to purchase** (from `URL` column)
  4. **A brief one-line key benefit**

- **Suggest four additional similar products**, different from the two main recommendations.
- **Format for product-related queries ONLY**:
  ```
  Here are some great options based on what you're looking for! 😊

  Recommended Products

  🔹 [Product Name] (₹[Price])
  👉 Buy Here : [URL]
  ✨ Key Benefit: [Brief one-line description]

  🔹 [Product Name] (₹[Price])
  👉 Buy Here : [URL]
  ✨ Key Benefit: [Brief one-line description]

  Other options you might like:

  ⭐ [Product Name] – ₹[Price] (Great for [differentiating feature])
  ⭐ [Product Name] – ₹[Price] (Perfect if you need [differentiating feature])
  ⭐ [Product Name] – ₹[Price] (Ideal for [differentiating feature])
  ⭐ [Product Name] – ₹[Price] (Loved for [differentiating feature])

  Let me know if you need more recommendations! 😊
  ```

#### **3️⃣ If "Out-of-Scope" Query:**
- Politely inform the user in a **warm and friendly** manner.
- Example response:
  ```
 Oh no! That’s a bit outside my expertise. But if you have any skincare, haircare, or health-related questions, I’d love to help! 😊

---
### **Final Instructions**
- **Do NOT mix formats** → Use **ONLY** the general query format for general queries and **ONLY** the product query format for product-related queries.
- **Ensure full text from all relevant columns is considered without truncation**.
- Provide a **clear, concise response** that **directly answers the user's query**.

---
User Query: "{user_query}"
Product Data: "{product_info}" (if applicable)


Now classify the query and respond accordingly.


"""),
    ("human", "User Query: {user_query}\n\nProduct Data:\n{product_info}")
])


In [ ]:

def generate_response(user_query, top_7_similar_df):
    product_info = format_product_descriptions(top_7_similar_df)

    messages = custom_prompt.format_messages(
        product_info=product_info,
        user_query=user_query
    )

    response = chat(messages)
    return response.content


# Output

In [ ]:
# response = generate_response(user_query, top_7_similar_df)
# print(response)


Oh no! That’s a bit outside my expertise. But if you have any skincare, haircare, or health-related questions, I’d love to help! 😊


In [ ]:
import textwrap

response = generate_response(user_query, top_7_similar_df)
wrapped_response = textwrap.fill(response, width=80)  # Adjust width as needed
print(wrapped_response)
